In [1]:
from rl_zoo3.train import train
import sys, argparse

sys.path.append('.')


# 사전 형식의 인자를 정의
args_dict = {
    "env": "PointMaze_UMaze-v3",
    "algo": "tqc",
    "wandb-project-name": "baselines",
    "track": False,
    "save-freq": 1000,
    "save-replay-buffer": True,
    "env-kwargs": {"continuing_task": False},
    "wandb-tags": ["maze", "no_continuing_task", "g095"],
    "hyperparams": {
        "n_timesteps": 1000000,
        "gamma": 0.95
    }
}

# 사전을 명령줄 인자 형태의 리스트로 변환
args = []
for key, value in args_dict.items():
    if isinstance(value, bool):
        # Boolean 값은 단순히 키만 추가
        if value:
            args.append(f'--{key}')
    elif isinstance(value, dict):
        # 사전 형식의 인자는 키:값 형태로 추가
        for sub_key, sub_value in value.items():
            args.append(f'--{key}')
            args.append(f'{sub_key}:{sub_value}')
    elif isinstance(value, list):
        # 리스트 형식의 인자는 키와 함께 값을 추가
        for item in value:
            args.append(f'--{key}')
            args.append(item)
    else:
        # 그 외의 경우 키와 값을 추가
        args.append(f'--{key}')
        args.append(str(value))

train(args)


/env2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


========== PointMaze_UMaze-v3 ==========
Seed: 2972468685
Loading hyperparameters from: /research/rl_zoo_jesnk/hyperparams/tqc.yml
Default hyperparameters for environment (ones being tuned will be overridden):
OrderedDict([('batch_size', 512),
             ('buffer_size', 1000000),
             ('gamma', 0.95),
             ('learning_rate', 0.001),
             ('learning_starts', 10000),
             ('n_timesteps', 1000000.0),
             ('normalize', False),
             ('policy', 'MultiInputPolicy'),
             ('policy_kwargs', 'dict(net_arch=[512, 512, 512], n_critics=2)'),
             ('replay_buffer_class', 'HerReplayBuffer'),
             ('replay_buffer_kwargs',
              "dict( goal_selection_strategy='future', n_sampled_goal=4, )"),
             ('tau', 0.005)])
Using 1 environments
Creating test environment
Using cuda device
HER REPLAY BUFFER
Log path: logs/tqc/PointMaze_UMaze-v3_6
---------------------------------
| rollout/           |          |
|    ep_len_m

In [ ]:
# Test code
'''
generate data like
trajectory = {
    "achieved_goal": self.observations["achieved_goal"][epi_index:epi_index+episode_length, env_indices, :],
    "actions": self.actions[epi_index:epi_index+episode_length, env_indices, :],
    "next_achieved_goal": self.next_observations["achieved_goal"][epi_index:epi_index+episode_length, env_indices, :],
    "desired_goal": self.observations["desired_goal"][epi_index:epi_index+episode_length, env_indices, :],
    "rewards": self.rewards[epi_index:epi_index+episode_length, env_indices, :],
    "dones": self.dones[epi_index:epi_index+episode_length, env_indices, :],
    "infos": self.infos[epi_index:epi_index+episode_length, env_indices, :],
    "timeouts": self.timeouts[epi_index:epi_index+episode_length, env_indices, :],
    "episode_index": epi_index,
}

'''
import numpy as np
trajectory = {}
# All data dim is (seq_length, 1 (n_envs), dim)
# state_dim is 2
# action_dim is 3

trajectory["achieved_goal"] = np.zeros(shape=(10, 1, 2))
trajectory["achieved_goal"][-1] = np.array([1, 1])
trajectory["actions"] = np.random.rand(10, 1, 3)
trajectory["next_achieved_goal"] = np.random.rand(10, 1, 2)
trajectory["desired_goal"] = np.random.rand(10, 1, 2)
trajectory["rewards"] = np.random.rand(10, 1, 1)
trajectory["dones"] = np.random.rand(10, 1, 1)
trajectory["infos"] = np.random.rand(10, 1, 1)
trajectory["timeouts"] = np.random.rand(10, 1, 1)
trajectory["episode_index"] = 0



trajectory["desired_goal"][:,:,:] = trajectory["achieved_goal"][-1, :, :] # must be test

trajectory["desired_goal"]

array([[[1., 1.]],

       [[1., 1.]],

       [[1., 1.]],

       [[1., 1.]],

       [[1., 1.]],

       [[1., 1.]],

       [[1., 1.]],

       [[1., 1.]],

       [[1., 1.]],

       [[1., 1.]]])

In [ ]:
from stable_baselines3.her.her_replay_buffer import CausalMultiHeadAttention

attention = CausalMultiHeadAttention(state_dim=2, max_len=20, embed_size=512, heads=8)
print(trajectory["achieved_goal"].shape)
trajectories = [trajectory, trajectory]
attention(trajectories)

(10, 1, 2)
episode_length :  10
queries shape : torch.Size([1, 1, 8, 64])
keys shape : torch.Size([10, 1, 8, 64])
attention shape : torch.Size([10, 8, 1, 1])
scalarized_output tensor([[ 0.2385],
        [-0.2165],
        [-0.1742],
        [ 0.1079],
        [ 0.2339],
        [-0.1317],
        [-0.2222],
        [-0.4834],
        [-0.2625],
        [ 0.0997]], grad_fn=<AddmmBackward0>)
tensor([0.6302, 0.6634, 0.6983, 0.7351, 0.7738, 0.8145, 0.8574, 0.9025, 0.9500,
        1.0000])
loss : tensor(0.8460, grad_fn=<MseLossBackward0>)


/env2/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


episode_length :  10
queries shape : torch.Size([1, 1, 8, 64])
keys shape : torch.Size([10, 1, 8, 64])
attention shape : torch.Size([10, 8, 1, 1])
scalarized_output tensor([[ 0.2385],
        [-0.2165],
        [-0.1742],
        [ 0.1079],
        [ 0.2339],
        [-0.1317],
        [-0.2222],
        [-0.4834],
        [-0.2625],
        [ 0.0997]], grad_fn=<AddmmBackward0>)
tensor([0.6302, 0.6634, 0.6983, 0.7351, 0.7738, 0.8145, 0.8574, 0.9025, 0.9500,
        1.0000])
loss : tensor(0.8460, grad_fn=<MseLossBackward0>)


[]